In [8]:
import pandas as pd
import numpy as np

from env import host, username, password, sql_connexion

import seaborn as sns
import scipy

from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

- Accuracy = (TP + TN) / (TP + FP + TN + FN)  
- Precision / PPV = TP / (TP + FP)  
- NVP / negative predicted value = TN / (TN + FN)
- Specificity = TN / (FP + TN)
- Recall / Sensitivity : TP / (TP + FN)

In [52]:
glasses = pd.DataFrame({
    'actual': ['yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes'],
    'prediction': ['yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'yes']
})
glasses

,actual,prediction
0,yes,yes
1,no,no
2,no,yes
3,yes,yes
4,yes,yes
5,yes,yes
6,no,no
7,yes,no
8,yes,yes
9,yes,no


### Confusion Matrix
  
A table of prediction vs actual.

In [53]:
pd.crosstab(glasses['actual'], glasses['prediction'])

prediction,no,yes
actual,,
no,2,2
yes,2,7


In [54]:
# from sklearn, get an array format of the crosstab table

sklearn.metrics.confusion_matrix(glasses['actual'], glasses['prediction'])

array([[2, 2],
       [2, 7]])

### Baseline  

All supervised machine-learning projects need a baseline to be determined, based on our data.  



In [55]:
glasses['actual'].value_counts()

# returns total nº yes / total nº no in confusion matrix

yes    9
no     4
Name: actual, dtype: int64

In [67]:
glasses['baseline'] = 'yes'
glasses

# talk about pc increase over the baseline, but the baseline can be 
# established (somewhat) according to one's determining

# choosing 'yes' bc it's the most numerous category in total values.

,actual,prediction,baseline
0,yes,yes,yes
1,no,no,yes
2,no,yes,yes
3,yes,yes,yes
4,yes,yes,yes
5,yes,yes,yes
6,no,no,yes
7,yes,no,yes
8,yes,yes,yes
9,yes,no,yes


### Accuracy  

- Out of all of our guesses, how many times we get it correct.  
( How many times did I guess correctly over how many times did I guess in total ?  )

-  (TP + TN) / (TP + FP + TN + FN) 

In [68]:
# calculate using 'glasses' df.

glasses['baseline'] == glasses['actual']

# Boolean baseline (T = 1, F = 0)

0      True
1     False
2     False
3      True
4      True
5      True
6     False
7      True
8      True
9      True
10    False
11     True
12     True
dtype: bool

In [69]:
baseline_accuracy = (glasses['baseline'] == glasses['actual']).mean()

# my baseline is relatively high : 69.23pc of guessing correctly

In [70]:
baseline_accuracy * 100

69.23076923076923

In [71]:
(glasses['prediction'] == glasses['actual']).mean()

# a 69pc accuracy in model prediction

0.6923076923076923

In [26]:
glasses = glasses.drop(columns = 'base')

### Precision  

Of all the times that I guess the positive case, how many times do I guess correctly ?

- Precision / PPV  =  TP / (TP + FP)  
- Precision is based on the positive case, but I'm the one who determines which is the positive case.

Every time that the model makes a FP prediction, it harms business. This is why precision is important.

In [72]:
# make a subset : all the element where we predicted CORRECTLY
# Baseline = 0

subset = glasses['prediction'] == 'yes'
subset

# Boolean series : Make a Boolean mask

0      True
1     False
2      True
3      True
4      True
5      True
6     False
7     False
8      True
9     False
10     True
11     True
12     True
Name: prediction, dtype: bool

In [73]:
# Boolean series : Make a Boolean mask

precision_df = glasses[subset]
precision_df

,actual,prediction,baseline
0,yes,yes,yes
2,no,yes,yes
3,yes,yes,yes
4,yes,yes,yes
5,yes,yes,yes
8,yes,yes,yes
10,no,yes,yes
11,yes,yes,yes
12,yes,yes,yes


In [74]:
(precision_df['prediction'] == precision_df['actual']).mean()

## this means that we predicted correctly 78pc of the times.

0.7777777777777778

### Recall

- Recall / Sensitivity : TP / (TP + FN)  
    
Of all actual positive cases, how many did I correctly identify ?  
FN is included bc, even though I guessed incorrectly, it was a positive case.

- Example : Caputure all possible positive events of hacking at a bank.
    - But, we could start to get more false positives (and your account gets locked when you forget your password).
    
    
    Use Recall when : in high-risk situations, want a high recall if don't want FP (ie, testing for cancer, coronavirus, etc, gives a FP, which is better than FN). 

In [75]:
recall_subset = glasses['actual'] == 'yes'

recall_subset

0      True
1     False
2     False
3      True
4      True
5      True
6     False
7      True
8      True
9      True
10    False
11     True
12     True
Name: actual, dtype: bool

In [76]:
## mask it with Boolean masking

recall_df = glasses[recall_subset]
recall_df

,actual,prediction,baseline
0,yes,yes,yes
3,yes,yes,yes
4,yes,yes,yes
5,yes,yes,yes
7,yes,no,yes
8,yes,yes,yes
9,yes,no,yes
11,yes,yes,yes
12,yes,yes,yes


In [77]:
recall_df['prediction' ] == recall_df['actual']

0      True
3      True
4      True
5      True
7     False
8      True
9     False
11     True
12     True
dtype: bool

In [78]:
(recall_df['prediction' ] == recall_df['actual']).mean()

# based on the actual values being 'yes' (cf Precision, which was the predicted values being 'yes')

0.7777777777777778

### Other Metrics

- Misclassification Rate : The rate of being inaccurate : 1 - accuracy.    
- Sensitivity : Detection of the positive class (aka, Recall). How well able to detect the positive case. Be careful of increased FPs.  
- Specificity : Correct detection of the negative class (ie, Recall for the negative class). 
- False Positive Rate : Rate at which model produces FPs.
- F1 Score : The harmonic mean of Precision & Recall.  
- AUC-ROC : Area under the receiver operating characteristic curve : Increase in rate of TP guesses also increases FP predictions.

### Multiclass Classification



In [79]:
sklearn.metrics.classification_report(glasses['actual'], glasses['prediction'])

'              precision    recall  f1-score   support\n\n          no       0.50      0.50      0.50         4\n         yes       0.78      0.78      0.78         9\n\n    accuracy                           0.69        13\n   macro avg       0.64      0.64      0.64        13\nweighted avg       0.69      0.69      0.69        13\n'

In [80]:
print(sklearn.metrics.classification_report(glasses['actual'], glasses['prediction']))

# two different ways of average : 
# macro  (average of 0.50 & 0.78), 
# weighted (skewed toward the most prevalent class in the classification, here, 'yes')

              precision    recall  f1-score   support

          no       0.50      0.50      0.50         4
         yes       0.78      0.78      0.78         9

    accuracy                           0.69        13
   macro avg       0.64      0.64      0.64        13
weighted avg       0.69      0.69      0.69        13

